# **Tarefa: Clustering**

### Nesta tarefa, você deve carregar um dataset e agrupar os objetos utilizando algoritmos de pelo menos duas categorias diferentes (centroide, hierárquico, densidade)

**Dicas:**
- Explore diferentes parâmetros, como o número de clusters ou a distância usada, para observar como isso afeta os resultados.
- Analise os insights que surgem a partir das visualizações dos agrupamentos encontrados.

# **Sobre o Dataset**

Este dataset contém informações detalhadas sobre clientes, como dados demográficos, hábitos de compra e interações com campanhas de marketing. Ele permite identificar perfis de clientes e realizar segmentações com base em suas preferências e comportamentos de consumo.

**Features:**
- **Year_Birth:** Ano de nascimento do indivíduo.
- **Education:** Nível mais alto de educação alcançado.
- **Marital_Status:** Estado civil do indivíduo.
- **Income:** Renda anual do indivíduo.
- **Kidhome:** Número de crianças pequenas na casa.
- **Teenhome:** Número de adolescentes na casa.
- **Recency:** Dias desde a última compra ou interação.
- **MntWines:** Quantia gasta em vinhos.
- **MntFruits:** Quantia gasta em frutas.
- **MntMeatProducts:** Quantia gasta em produtos de carne.
- **MntFishProducts:** Quantia gasta em produtos de peixe.
- **MntSweetProducts:** Quantia gasta em doces.
- **NumStorePurchases:** Número de compras em lojas físicas.
- **NumWebVisitsMonth:** Número de visitas ao site da empresa em um mês.


# **Carregue as bibliotecas**

Dica: Para esta tarefa, além das bibliotecas `pandas` e `matplotlib`, explore funções das bibliotecas `sklearn` e `sklearn_extra`. Considere os seguintes pacotes:
- **cluster:** KMeans, KMedoids, AgglomerativeClustering, DBSCAN
- **preprocessing:** MinMaxScaler (normalização) e OneHotEncoder (binarização)
- **metrics:** silhouette_score

Sinta-se à vontade para adicionar outros pacotes que achar úteis.

```python
!pip install scikit-learn-extra
from sklearn_extra.cluster import KMedoids
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN


### **Carregue os Dados**

## **Pré-Processamento dos Dados**

Aplique as técnicas de pré-processamento que vimos em aula no dataset fornecido.

### **Dicas:**
- **Converter dados categóricos para numéricos:** Use `LabelEncoder` ou `OneHotEncoder`.

- **Transformar os dados:** Experimente `MinMaxScaler` para ajustar a escala dos dados.

- **Remover outliers:** Verifique outliers com base no IQR e os elimine se necessário.

- **Tratar valores nulos:** Preencha com a média, moda, mediana ou remova as linhas, dependendo do impacto no dataset.


# **Clustering**

**Escolha ao menos 2 algoritmos de clustering que pertençam a métodos diferentes (centróides, densidade ou hierárquico).**

### **Passos:**

1. Para cada algoritmo, execute-o em 6 diferentes cenários, variando os parâmetros.

2. Calcule o índice de silhueta `silhouette_score` para cada cenários e compare os resultados.

### **Resultados**

1. **Resumo dos Resultados:**
   - Para cada algoritmo utilizado, apresente os resultados dos 6 cenários testados e compare as métricas calculadas.

2. **Visualização Gráfica:**
   - Plote um gráfico comparando o índice de silhueta entre os cenários.





## **Encontre o melhor clustering para cada um dos algoritmos selecionados**

Depois de executar os algoritmos com diferentes combinações de parâmetros, identifique quais obtiveram os melhores resultados de acordo com índice de silhueta para cada um dos 2 (ou mais) algoritmos selecionados.


## **Para cada algoritmo, visualize graficamente o modelo que teve o melhor desempenho**

Com base nos modelos de melhor desempenho, escolha variáveis e crie gráficos de dispersão que permitam visualizar os clusters gerados.

**Dica:**

Ao criar scatter plots, teste combinações de 2 variáveis do dataset para facilitar a visualização dos clusters. Para o clustering hierárquico, experimente também plotar um dendrograma.

### **Dendrograma:**
Use o código abaixo para o dendrograma:
```python
import scipy.cluster.hierarchy as sch

Z = sch.linkage(df[[variavel_1, variavel_2]], method='centroid')

# p = altura máxima que será exibida
sch.dendrogram(Z, p = ?, truncate_mode = "level") # Mude o valor de p para melhorar a vizualização do seu dendrograma

```



[
Documentação Dendrograma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html#scipy.cluster.hierarchy.dendrogram)

### **Análise dos Clusters**

Após identificar o melhor modelo, selecione **3 itens de cada cluster** e explique o que cada agrupamento parece representar com base nas características dos itens.

1. **Análise dos Itens:**
   - Analise as características principais desses itens (ex.: renda, número de compras, produtos comprados).
   - Identifique padrões ou características que diferenciam cada cluster.

2. **Interpretação dos Agrupamentos:**
   - Explique o que cada cluster parece estar representando.
   - Relacione as características predominantes de cada cluster a possíveis perfis de clientes (ex.: clientes com alta renda e foco em produtos premium, ou clientes com baixa frequência de compras).


Responda aqui....